## Utility functions

In [ ]:
#! pip install geopy
#! pip install csv
#! pip install itertools
#!pip install random

In [ ]:
import pandas as pd
import os
import time
import copy
import shutil
from datetime import datetime
import scipy.integrate as integrate
from scipy.stats import expon
import numpy as np
import yaml
import pandas as pd
from pathlib import Path
import yaml
from geopy.distance import distance
import time
import csv
from pathlib import Path
import itertools as it
from sklearn.metrics import mean_squared_error
import random
from scipy.spatial.distance import cityblock
import pickle


import math
global conf
ROOT = Path().resolve()
    


In [ ]:
def convert(s):
    try:
        return float(s)
    except ValueError:
        num, denom = s.split('/')
        return float(num) / float(denom)

def append_to_csv(file_path, data):
    dir = os.path.dirname(file_path)
    if not os.path.exists(dir):
        os.makedirs(dir)
    if not os.path.exists(file_path):
        # If the file doesn't exist, create a new file
        data.to_csv(file_path, index=True)
    else:
        # If the file exists, append the data to it
        data.to_csv(file_path, index=True, mode='a', header=False)

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    # total_meters = METERS * c
    r = 6371000  # radiu * 1000 to return meters
    return c * r



def calculate_coverage(min_distance, scale, detour_radius):
    # initialize coverage inner function
    #return integrate.quad(lambda x: expon.pdf(x, scale=scale), min_distance, np.inf)[0]

    ## Truncate of the Exponential Distribution
    return integrate.quad(lambda x: expon.pdf(x, scale=scale), min_distance, np.inf)[0]/(1-math.exp(-scale*detour_radius))

# clean all the files inside the run path
def clean_run_path(run_path):
    if(Path(run_path).exists()):
        shutil.rmtree(run_path)
        Path(run_path).mkdir(parents=True, exist_ok=True)
        Path(run_path+"cov_series").mkdir(parents=True, exist_ok=True)
    else:
        Path(run_path).mkdir(parents=True, exist_ok=True)
        Path(run_path+"cov_series").mkdir(parents=True, exist_ok=True)

#MANFREDO calculate euclidean distance and manhattan distance between coverage map final and node's coverage
def euclidean_distance(covmap):
    df_final = pd.read_csv(path_covmap)
    probability_final = np.array(df_final.sort_values(by="id_location")['probability'])
    cov_to_test = np.array(covmap.sort_index(axis=0)["probability"])
    
    return np.linalg.norm(probability_final - cov_to_test)

def manhattan_distance(covmap):
    df_final = pd.read_csv(path_covmap)
    probability_final = np.array(df_final.sort_values(by="id_location")['probability'])
    cov_to_test = np.array(covmap.sort_index(axis=0)["probability"])

    return cityblock(probability_final,cov_to_test)

class Node:

    def __init__(self, uid, pois, scale, detour_radius, ageing):
        self.uid = uid
        self.POIs = pois
        self.scale = scale
        self.ageing = ageing
        self.detour_radius = detour_radius
        #self.log = []
        #self.merge_log = []  # Keeps track of the data exchange with other nodes
        self.cov_count = 0
        self.coverage = pd.DataFrame(columns=["probability"])
        #self.coverage_map = pd.read_csv("swap_covmap_real2.csv")
        self.coverage_map = pd.read_csv(path_covmap)
        #self.backup_coverages = []
        self.current_trajectory = pd.DataFrame(columns=["lat", "lon", "tid"])
        self.adjacency = {}
        self.recent_exchanges = []
        self.ageing_start = {}
        self.black_list = {}
        self.filter_threshold = 0

    def move(self, lat, lon, tid, bid):
        if self.current_trajectory.size != 0:

            if self.current_trajectory.loc[0].tid != tid:
                #print("--> Computing node coverage map")
                start_coverage = time.perf_counter()
                self.compute_coverage_map(bid)
                end_coverage = time.perf_counter()
                #self.log.append((self.uid, end_coverage - start_coverage))
                #print(f"--> Coverage map computed in {end_coverage - start_coverage:0.4f}")

                # forse togliere
                # self.trajectories.append(self.current_trajectory)

                self.current_trajectory = pd.DataFrame(columns=["lat", "lon", "tid"])

            self.current_trajectory.loc[len(self.current_trajectory)] = [lat, lon, tid]

        else:
            self.current_trajectory.loc[0] = [lat, lon, tid]

    def compute_coverage_map(self, bid):
        # Michele : aggiunto check
        if self.ageing:
            self.do_age_coverage(bid)

        for index, location in self.POIs.iterrows():

            d = min(
                haversine(location.lat,
                                   location.lon,
                                   self.current_trajectory['lat'].values,
                                   self.current_trajectory['lon'].values
                                   )
            )

            #attack PASL
            
            if self.uid not in attackers:
                    #coverage_probability = 0.0
                    #coverage_probability = random.random()
                if d <= self.detour_radius:
                    coverage_probability = calculate_coverage(d, self.scale, self.detour_radius)
                else:
                    coverage_probability = 0.0
            else:
                coverage_probability = 0.0
            
            
            #random attack
            '''
            if self.uid not in attackers:
                if d <= self.detour_radius:
                    coverage_probability = calculate_coverage(d, self.scale, self.detour_radius)
                else:
                    coverage_probability = 0.0
            else:
                coverage_probability = random.random()
            '''    
            
            if location.id_location in self.coverage.index:
                #print("probability_union", self.coverage.loc[location.id_location][0], coverage_probability)
                if self.uid not in attackers:
                    self.coverage.loc[location.id_location] = [probability_union(self.coverage.loc[location.id_location][0], coverage_probability)]
                else:
                    self.coverage.loc[location.id_location] = coverage_probability   #attack PASL commentate queste due
            else:
                coverage_to_add = pd.DataFrame(coverage_probability, columns=["probability"], 
                                         index=[location.id_location])
                self.coverage = pd.concat([self.coverage, coverage_to_add])

            #print("Coverage for location {} and node {} calculated successfully with value: {} and distance {}".format(
                #location.id_location,
                #self.uid,
                #self.coverage.loc[location.id_location].probability,
                #d)
            #)

            if location.id_location in self.ageing_start.keys():
                if coverage_probability > 0:
                    self.ageing_start[index] = bid
            else:
                self.ageing_start[location.id_location] = bid

        # SAVE COPY OF EVERY COVERAGE MAP
        cov_copy = self.coverage.copy()
        cov_copy['copy_id'] = self.cov_count
        cov_copy['bid'] = bid

        append_to_csv(run_path+"cov_series/"+str(self.uid)+"_coverages.csv", cov_copy)
        #self.backup_coverages.append(cov_copy)
        self.cov_count += 1

        # RESTORE RECENT EXCHANGES LIST
        self.recent_exchanges = []

        #inizialize filter threshold
        #if self.filter_threshold == 0:
        self.filter_threshold = manhattan_distance(self.coverage)
        #self.filter_threshold = euclidean_distance(self.coverage)
        #check if there are some users in blacklist with a lower manhattan distance then the new threshold
        keys_to_remove = []
        for user in self.black_list.keys():
            if self.black_list[user] <= self.filter_threshold:
                print("NEW THRESHOLD after COMPUTE COVMAP, RIMOZIONE DALLA BL DI:",user)
                keys_to_remove.append(user)
        for node in keys_to_remove:
            del self.black_list[node]
                

    def do_age_coverage(self, bid):

        for location in self.coverage.index:

            if self.coverage.loc[location][0] != 0:
                ageing_percentage = power_law(start=self.ageing_start[location], current=bid)

                if ageing_percentage < 1:
                    self.ageing_start[location] = bid
                    self.coverage.loc[location] = self.coverage.loc[location] * ageing_percentage
    '''
    def merge_coverage(self, bid):

        if self.ageing:
            self.do_age_coverage(bid)

        for node in self.adjacency.keys():

            if node not in self.recent_exchanges:

                # merge current coverage map with all the nodes of the adjacency list and remove current node from the in o
                # order to ovid multiple, useless, merge operations
                if self.ageing:
                    self.adjacency[node].do_age_coverage(bid)

                cov, ageing = merge(self.coverage, self.adjacency[node].coverage, self.ageing_start,
                                          self.adjacency[node].ageing_start)
                self.coverage = cov.copy()
                self.adjacency[node].coverage = cov
                self.ageing_start = copy.deepcopy(ageing)
                self.adjacency[node].ageing_start = ageing

                # Since at every time bucket the position of a node is the result of an average of the points of its trajectory,
                # it could be possible that a node x has another node y in his adjacency list but not vice versa
                try:
                    self.adjacency[node].recent_exchanges.append(self.uid)
                    del self.adjacency[node].adjacency[self.uid]
                except KeyError:
                    continue
                finally:
                    self.recent_exchanges.append(node)
                    self.merge_log.append((self.uid, node, bid))

        self.adjacency = {}
'''        
                  

def probability_union(p1, p2):
    # P(AUB) = P(A) + P(B) - P(A)*P(B)
    return 1 - (1 - p1) * (1 - p2)

'''
def merge(cov1, cov2, ageing1, ageing2):
    for index, row in cov1.iterrows():

        if index in cov2.index:

            cov2.at[index] = probability_union(row.probability, cov2.at[index, 'probability'])
            ageing2[index] = max(ageing2[index], ageing1[index])

        else:
            cov2 = cov2.append(pd.DataFrame(row.probability, columns=["probability"],
                                            index=[index]))
            ageing2[index] = ageing1[index]

    return cov2, ageing2
'''

def power_law(start, current):
    days = int(current - start) / 1000000000 / 60 / 60 / 24

    if days == 0:
        return 1

    res = conf['powerlaw_a_param'] * pow(days, conf['powerlaw_k_param'])

    if res >= 1:
        return 1
    else:
        return res
        
#filter with manhattan distance

def filter_check(node_a_uid, node_b_uid, manhattan_a, manhattan_b):
    if (manhattan_a <= nodes[node_b_uid].filter_threshold ) and (manhattan_b <= nodes[node_a_uid].filter_threshold):
        return True
    #check if node_a blacklist is full, check if the new Manhattan is the lowest in the blacklist, in that case return TRUE 
    elif (manhattan_b > nodes[node_a_uid].filter_threshold) and (manhattan_a <= nodes[node_b_uid].filter_threshold):
        if len(nodes[node_a_uid].black_list) == max_blacklist:
            #print("FILTER CHECK NODE A: ", node_a_uid)
            check = True
            for key in nodes[node_a_uid].black_list.keys():
                if nodes[node_a_uid].black_list[key] < manhattan_b:
                    check = False
            if check == True:
                nodes[node_a_uid].filter_threshold = manhattan_b
                return True
    #check if node_b blacklist is full, check if the new Manhattan is the lowest in the blacklist, in that case return TRUE 
    elif (manhattan_a > nodes[node_b_uid].filter_threshold) and (manhattan_b <= nodes[node_a_uid].filter_threshold):
        if len(nodes[node_b_uid].black_list) == max_blacklist:
            #print("FILTER CHECK NODE B: ", node_b_uid)
            check = True
            for key in nodes[node_b_uid].black_list.keys():
                if nodes[node_b_uid].black_list[key] < manhattan_a:
                    check = False
            if check == True:
                nodes[node_b_uid].filter_threshold = manhattan_a
                return True
    else:
        return False

#filter with euclidean distance
'''
def filter_check(node_a_uid, node_b_uid, euclidean_a, euclidean_b):
    #if nodes[node_b_uid].filter_threshold == 0 or nodes[node_a_uid].filter_threshold == 0:
    #    return True
    #if (node_b_uid in nodes[node_a_uid].black_list.keys()) or (node_a_uid in nodes[node_b_uid].black_list.keys()):
    #    return False
    if (euclidean_a <= nodes[node_b_uid].filter_threshold ) and (euclidean_b <= nodes[node_a_uid].filter_threshold):
        return True
    #check if node_a blacklist is (max - 1), check if the new euclidean is the lowest in the blacklist, in that case return TRUE 
    elif (euclidean_b > nodes[node_a_uid].filter_threshold) and (euclidean_a <= nodes[node_b_uid].filter_threshold):
        if len(nodes[node_a_uid].black_list) == max_blacklist-1:
            #print("FILTER CHECK NODE A: ", node_a_uid)
            check = True
            for key in nodes[node_a_uid].black_list.keys():
                if nodes[node_a_uid].black_list[key] < euclidean_b:
                    check = False
            if check == True:
                nodes[node_a_uid].filter_threshold = euclidean_b
                return True
    #check if node_b blacklist is (max - 1), check if the new euclidean is the lower in the blacklist, in that case return TRUE 
    elif (euclidean_a > nodes[node_b_uid].filter_threshold) and (euclidean_b <= nodes[node_a_uid].filter_threshold):
        if len(nodes[node_b_uid].black_list) == max_blacklist-1:
            #print("FILTER CHECK NODE B: ", node_b_uid)
            check = True
            for key in nodes[node_b_uid].black_list.keys():
                if nodes[node_b_uid].black_list[key] < euclidean_a:
                    check = False
            if check == True:
                nodes[node_b_uid].filter_threshold = euclidean_a
                return True
    else:
        return False
'''
#MANFREDO attack 2, attackers receive a covmap with a swap and change probability value from this one
def poisoning_data(covmap):
    for index, row in covmap.iterrows():
        if row.probability > 0.5:
            #print(str(index)+" prima: "+str(row.probability)+" --> dopo: 0.01")
            covmap.at[index,'probability'] = 0.01
        elif row.probability > 0:
            #print(str(index)+" prima: "+str(row.probability)+" --> dopo: 0.99")
            covmap.at[index,'probability'] = 0.99
    return covmap

def check_threshold(uid, blacklist):
    min_manhattan = 1000
    #min_euclidean = 50
    keys = blacklist.keys()
    keys_to_remove = []
    print("CHECK THRESHOLD NODE: ",uid)
    print("NODE IN BL: ", keys)
    print("OLD THRESHOLD: ", nodes[uid].filter_threshold)
    for key in keys:
        if blacklist[key] < min_manhattan:
        #if blacklist[key] < min_euclidean:    
            min_manhattan = blacklist[key]
            #min_euclidean = blacklist[key]
    nodes[uid].filter_threshold = min_manhattan
    #nodes[uid].filter_threshold = min_euclidean
    for k in keys:
        #if blacklist[k] == min_euclidean:
        if blacklist[k] == min_manhattan:
            keys_to_remove.append(k)
    print("RIMOZIONI DALLA BL DEI NODI: ", keys_to_remove)
    for node in keys_to_remove:
        del nodes[uid].black_list[node]
    
    print("NEW THRESHOLD: ", min_manhattan)

#final clean of all blacklists before sending them to the server
def clean_blacklists():
    for node in nodes.keys():
        keys = nodes[node].black_list.keys()
        keys_to_remove = []
        for key in keys:
            if nodes[node].black_list[key] <= nodes[node].filter_threshold:
                keys_to_remove.append(key)
        for uid in keys_to_remove:
            del nodes[node].black_list[uid]




## Simulator

In [ ]:
log = []
nodes = {}
time_buckets = []



def initialize(path_gps, contacts, granularity=None, start_date=None, end_date=None, synthetic=False):
    print("Initialization started...")
    print("Reading:",path_gps)
    geo = pd.read_csv(path_gps, parse_dates = True)

    print("-> Filtering geolife")

    geo['date_time'] = pd.to_datetime(geo['date_time'], errors='coerce')

    # fiilter with the dates
    geo = geo[(geo.date_time >= start_date) & (geo.date_time <= end_date)]

    # filter with the bounding box
    geo = geo[(geo['lat'].between(beijing_lat_min, beijing_lat_max)) & (geo['lon'].between(beijing_lon_min, beijing_lon_max))]
    
    print(len(geo))
    
    
    if granularity == "min":
        geo['date_time'] = geo['date_time'].dt.floor('min')
    if granularity == "h":
        geo['date_time'] = geo['date_time'].dt.floor('h')
    if granularity == "sec":
        geo['date_time'] = geo['date_time'].dt.floor('s')

    users = geo['uid'].unique()
    max_blacklist = np.round((percentage_attackers*len(users))/100)
    print("MAX BLACKLIST: ", max_blacklist)
    
    geo = geo.groupby(['date_time', 'uid'], as_index=False).mean()

    for time in geo['date_time'].unique():
        time_buckets.append(geo[geo['date_time'] == time])

    # create contacts dataframe
    print("Creating contacts file")
    uids = set(geo["uid"])
    dfs_to_add = [
        pd.DataFrame({"node_a": [node_a], "node_b": [node_b], "contacts": [0]})
        for node_a, node_b in it.combinations(uids, 2)
    ]
    # keep it in RAM
    contacts = pd.concat(dfs_to_add, ignore_index=True)
    #contacts.to_csv(run_path+"contacts"+"_"+str(scale)+"_"+str(detour_radius)+".csv", index=False)
        
    print("Initialization ended:",len(time_buckets))
    return contacts, max_blacklist

def start_simulation(path_pois, scale, detour_radius, contacts, run_path):
    print("Reading POIs...")
    pois = pd.read_csv(path_pois)
    #MANFREDO create res to save swaps and statistics for each swap
    res=[]
    filter=[]
    
    #filter POIs according to the Beijing
    print("number of locations:",len(pois))
    
    counter = 1
    for bucket in time_buckets:
        #print(bucket["date_time"].unique())
        #start_bucket = time.perf_counter()

        inner_counter = 1
        for c_index, current_node in bucket.iterrows():
            #print("--> Processing node {} of {}".format(inner_counter, len(bucket)))
            inner_counter += 1
            #print(str(inner_counter))
            if current_node.uid not in nodes.keys():
                ageing = conf["ageing_active"]
                print("CREAZIONE NODO: ", current_node.uid)
                nodes[current_node.uid] = Node(current_node.uid, pois, scale, detour_radius, ageing)
                nodes[current_node.uid].move(lat=current_node.lat, lon=current_node.lon, tid=current_node.tid,
                                             bid=bucket['date_time'].values[0])
            else:
                nodes[current_node.uid].move(lat=current_node.lat, lon=current_node.lon, tid=current_node.tid,bid=bucket['date_time'].values[0])

            if conf["ADJACENCY_THRESHOLD"] > 0:
                for node in nodes:
                    if current_node.uid == node: continue
                    node_b = node
                    #idx = contacts.index[(contacts["node_a"]==node_a) & (contacts["node_b"] == node_b)]
                    #r = contacts.loc[idx]
                    #if r.empty:
                        #contacts = pd.concat([contacts, pd.Series([node_a,node_b,0])], ignore_index=True)
                    #    contacts = contacts.append({"node_a":node_a,"node_b":node_b,"contacts":0},ignore_index=True)
                    #print("--> Computing nodes adjacency")
                    #start_adjacency = time.perf_counter()

                    # Calculate distance between two nodes
                    d = distance((float(current_node.lat),
                                  float(current_node.lon)),
                                 (float(nodes[node_b].current_trajectory.tail(1).lat.iloc[0]),
                                  float(nodes[node_b].current_trajectory.tail(1).lon.iloc[0])))
                    
                    if d.meters <= conf['ADJACENCY_THRESHOLD']:
                        #idx = contacts.index[(contacts["node_a"]==node_a) & (contacts["node_b"] == node_b)]
                        #r = contacts.loc[idx]
                        #contacts.loc[idx,['contacts']] = r["contacts"]+1
                        #print("contact:",current_node.uid,nodes[neighbor])
                        filtered_rows = contacts[
                        ((contacts["node_a"] == current_node.uid) & (contacts["node_b"] == node_b)) |
                        ((contacts["node_a"] == node_b) & (contacts["node_b"] == current_node.uid))
                        ]
                        contacts.loc[filtered_rows.index, "contacts"] += 1                   

                        #MANFREDO count number of POI with non zero probability
                        node_a_POI = np.count_nonzero(nodes[current_node.uid].coverage)
                        node_b_POI = np.count_nonzero(nodes[node_b].coverage)
                        
                        #calculate euclidean and manhattan distance between coverage and final coverage map
                        if node_a_POI > 0:
                            euclidean_a = euclidean_distance(nodes[current_node.uid].coverage)
                            manhattan_a = manhattan_distance(nodes[current_node.uid].coverage)
                        else:
                            euclidean_a = 0
                            manhattan_a = 0
                        if node_b_POI > 0:
                            euclidean_b = euclidean_distance(nodes[node_b].coverage)
                            manhattan_b = manhattan_distance(nodes[node_b].coverage)
                        else:
                            euclidean_b = 0
                            manhattan_b = 0
                        
                        res.append([int(current_node.uid), int(node_b), node_a_POI, node_b_POI, euclidean_a, euclidean_b, manhattan_a, manhattan_b,nodes[current_node.uid].filter_threshold,nodes[node_b].filter_threshold])
                        # filtro per accettare lo scambio
                        if filter_active == True:
                            if filter_check(current_node.uid, node_b, manhattan_a, manhattan_b) == True:
                            #if (filter_check(current_node.uid, node_b, euclidean_a, euclidean_b) == True):
                                #caso scambio valido
                                #print("SWAP EFFETTUATO")
                                #if euclidean_a == 0 and euclidean_b==0:
                                if manhattan_a == 0 and manhattan_b==0:
                                    filter.append([int(current_node.uid), int(node_b), 1, 1])
                                elif (current_node.uid in attackers and node_a_POI > 0) or (node_b in attackers and node_b_POI > 0):
                                    filter.append([int(current_node.uid), int(node_b), 1, 0])
                                else:
                                    filter.append([int(current_node.uid), int(node_b), 1, 1])
                                #nodes[current_node.uid].adjacency[neighbor] = nodes[neighbor]
                                old_coverage = nodes[current_node.uid].coverage.copy()
                                
                                #attackers don't take the other's covmap (random attack)
                                #if current_node.uid not in attackers:
                                #    nodes[current_node.uid].coverage = nodes[node_b].coverage.copy()
                                #if node_b not in attackers:
                                #    nodes[node_b].coverage = old_coverage

                                #attack2.0
                                nodes[current_node.uid].coverage = nodes[node_b].coverage.copy()
                                nodes[node_b].coverage = old_coverage
                                
                                #initialize threshold, if are 0, with coverage map received , if not 0, threshold take the manhattan value from the coverage map received
                                #euclidean
                                #if nodes[current_node.uid].filter_threshold == 0 and euclidean_b !=0 :
                                #    nodes[current_node.uid].filter_threshold = euclidean_distance(nodes[current_node.uid].coverage) 
                                #if nodes[node_b].filter_threshold == 0 and euclidean_a != 0:
                                #    nodes[node_b].filter_threshold = euclidean_distance(nodes[node_b].coverage)
                                #manhattan    
                                if nodes[current_node.uid].filter_threshold == 0 and manhattan_b !=0 :
                                    nodes[current_node.uid].filter_threshold = manhattan_distance(nodes[current_node.uid].coverage) 
                                if nodes[node_b].filter_threshold == 0 and manhattan_a != 0:
                                    nodes[node_b].filter_threshold = manhattan_distance(nodes[node_b].coverage)
                                
                                
                                #attack 2.0
                                if node_b in attackers:
                                    nodes[node_b].coverage = poisoning_data(nodes[node_b].coverage)
                                elif current_node.uid in attackers:
                                    nodes[current_node.uid].coverage = poisoning_data(nodes[current_node.uid].coverage)
                            else:
                                #print("SWAP NON EFFETTUATO")
                                if (current_node.uid in attackers) or (node_b in attackers):
                                    filter.append([int(current_node.uid), int(node_b), 0, 0])
                                else:
                                    filter.append([int(current_node.uid), int(node_b), 0, 1])
                                #blacklist append here
                                #if (euclidean_a != 0 and euclidean_a > nodes[node_b].filter_threshold):
                                if (manhattan_a != 0 and manhattan_a > nodes[node_b].filter_threshold):
                                    #node_a probably is an attacker, append node_a in node_b's black list
                                    if current_node.uid not in nodes[node_b].black_list:
                                        #nodes[node_b].black_list[int(current_node.uid)] = euclidean_a
                                        nodes[node_b].black_list[int(current_node.uid)] = manhattan_a
                                        if len(nodes[node_b].black_list) == max_blacklist:
                                            check_threshold(node_b,nodes[node_b].black_list)     #aggiornamento threshold e rimozione utente con distanza minore
                                #elif (euclidean_b != 0 and euclidean_b > nodes[current_node.uid].filter_threshold):
                                elif (manhattan_b != 0 and manhattan_b > nodes[current_node.uid].filter_threshold):
                                    #node_b probably is an attacker, append node_b in node_a's black list
                                    if node_b not in nodes[current_node.uid].black_list:
                                        #nodes[current_node.uid].black_list[int(node_b)] = euclidean_b
                                        nodes[current_node.uid].black_list[int(node_b)] = manhattan_b
                                        if len(nodes[current_node.uid].black_list) == max_blacklist:
                                            check_threshold(current_node.uid,nodes[current_node.uid].black_list) #aggiornamento threshold e rimozione utente con distanza minore
                                   
                                #print("after",nodes[node_a].coverage.head()) 
                        else:
                            #filter not active, swap
                            old_coverage = nodes[current_node.uid].coverage.copy()   
                            nodes[current_node.uid].coverage = nodes[node_b].coverage.copy()
                            nodes[node_b].coverage = old_coverage
                            #attack PASL without filter
                            '''
                            if node_b in attackers:
                                #print("ATTACKER: "+str(node_b))
                                nodes[node_b].coverage = poisoning_data(nodes[node_b].coverage)
                            if current_node.uid in attackers:
                                #print("ATTACKER: "+str(current_node.uid))
                                nodes[current_node.uid].coverage = poisoning_data(nodes[current_node.uid].coverage)
                           '''

                #end_adjacency = time.perf_counter()
                #print(f"--> Node adjacency computed in {end_adjacency - start_adjacency:0.4f}")
                #log.append((bucket['date_time'].values[0], "adjacency", end_adjacency - start_adjacency))

        
        #end_bucket = time.perf_counter()
        #print("-> Bucket analyzed in {}".format(end_bucket - start_bucket))

        #print("-> Merging coverages map") if len(nodes.keys()) > 0 else print("-> No merge")
        #start_merging = time.perf_counter()
#         for node in nodes.keys():
#             if nodes[node].adjacency != {}:
#                 print("node:",nodes[node].uid)
#                 nodes[node].swap_coverage(bid=bucket['date_time'].values[0])
        #end_merging = time.perf_counter()
        #log.append((bucket['date_time'].values[0], "merge", end_merging - start_merging))
        #print(f"-> Merge procedure completed in {end_merging - start_merging:0.4f}\n")
        counter += 1
    #MANFREDO saving statistics about swaps and filter
    if conf["ADJACENCY_THRESHOLD"] > 0:
        df_POI_covered = pd.DataFrame(res,columns=['node_a','node_b','POI_node_a','POI_node_b', 'ED_a', 'ED_b', 'MH_a', 'MH_b', 'Threshold_a', 'Threshold_b'])
        df_POI_covered.to_csv(run_path+"swaps_"+str(scale)+"_"+str(detour_radius)+".csv", index=True)
        df_filter = pd.DataFrame(filter, columns=['node_a','node_b','filter','true_label'])
        df_filter.to_csv(run_path+"filter_"+str(scale)+"_"+str(detour_radius)+".csv", index=True)
    #return contacts


#def finalize_simulation(run_path):
#    bid = time_buckets[-1]['date_time'].values[0]
#    for node in nodes.keys():
#        nodes[node].compute_coverage_map(bid=bid)
        #print("nodes raw",nodes[node].coverage.probability.values)
    #for node in nodes.keys():
    #    start_merging = time.perf_counter()
    #    nodes[node].merge_coverage(bid=bid)
    #    end_merging = time.perf_counter()
    #    log.append(("", "merge", end_merging - start_merging))
    #    print("merged",nodes[node].coverage.probability.values) 
#        pd.concat(nodes[node].backup_coverages).to_csv(run_path+"cov_series/"+str(node)+"_coverages.csv")


def save_results(run_path, scale, detour_radius,contacts):
    print("Collecting coverages...")
    res = []
    #logs = []
    #merge_logs = []
    for node in nodes.keys():
        print("SALVATAGGIO COVERAGE NODO: ", node)
        df_node_with_uid = pd.DataFrame()
        df_node_with_uid['uid'] = [node] * len(nodes[node].coverage)
        df_node_with_uid['id_location'] = nodes[node].coverage.index
        df_node_with_uid['probability'] = nodes[node].coverage.probability.values
        df_node_with_uid['scale'] = scale
        df_node_with_uid['detour_radius'] = detour_radius
        
        res.append(df_node_with_uid)
        print("COVERAGE MAP: ",nodes[node].coverage)
        #print("RES: ",res)
     #   logs += nodes[node].log
     #   merge_logs += nodes[node].merge_log

    print("Saving results...")
    pd.concat(res).to_csv(run_path+"coverage_"+str(scale)+"_"+str(detour_radius)+".csv")
    # Print the contact list only if in swap mode
    if ADJACENCY_THRESHOLD > 0:
        print(contacts)
        contacts.to_csv(run_path+"contacts_"+str(scale)+"_"+str(detour_radius)+".csv")
        #create dictionary for blacklists and save it in a file
        dict_blacklist = {}
        clean_blacklists() #clean blacklist before submit to the server
        for node in nodes:
            node_blacklist = []
            for key in nodes[node].black_list.keys():
                node_blacklist.append(key)
            dict_blacklist[node] = node_blacklist
        with open(run_path+"blacklists_"+str(scale)+"_"+str(detour_radius)+".pk1", "wb") as fp:
            pickle.dump(dict_blacklist, fp)
            print('blacklist saved successfully to file')
            print(dict_blacklist)
#     with open(run_path+'log_coverage.csv', 'w') as out:
#         csv_out = csv.writer(out)
#         csv_out.writerow(['node', 'time'])
#         for row in logs:
#             csv_out.writerow(row)

#     with open(run_path+"log_merge.csv", 'w') as out:
#         csv_out = csv.writer(out)
#         csv_out.writerow(['node_1', 'node_2', 'bid'])
#         for row in merge_logs:
#             csv_out.writerow(row)

## Execute the model

In [ ]:
with open(f"{ROOT}/conf.yaml") as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)

cov_series_path = conf["cov_series_path"]
start_date=conf['start_date']
ends =conf['end_date']
path_gps = conf['path_gps']
out_path = conf["out_path"]
path_pois = conf["path_pois"]
granularity=conf['granularity']
scales = conf["scale_values"]
detour_rs = conf["detour_radius"]
beijing_lat_min = conf["beijing_lat_min"]
beijing_lat_max = conf["beijing_lat_max"]
beijing_lon_min = conf["beijing_lon_min"]
beijing_lon_max = conf["beijing_lon_max"]
ADJACENCY_THRESHOLD = conf["ADJACENCY_THRESHOLD"]
filter_active = conf["filter_active"]
#max_blacklist = conf["max_blacklist"]
percentage_attackers = conf["perc_max_attackers"]
attack_path = conf["attack_path"]
path_covmap = conf["path_covmap"]
print(conf)

max_blacklist = 0
attackers = [128,30,140,84]
#attackers = [30,128]
#attackers = []
for end_date in ends:
    print("Period:",end_date)
    if(int(ADJACENCY_THRESHOLD)>0):
        out_path+="swap/no_filter/"

    if not attackers:
        run_path = out_path+datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")+"_"+datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")+"/"
    else:
        run_path = attack_path+datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")+"_"+datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")+"/"
    clean_run_path(run_path)
    for scale in scales:
        for detour_radius in detour_rs:
            contacts = pd.DataFrame(columns=["node_a","node_b","contacts"])
            print("Simulating with:",scale,detour_radius)
            print("Number of attackers: ", len(attackers))
            ## MICHELE: PARAMETRIZZARE SCALE; D_RADIUS
            #initialize(path_gps,granularity,start_date, end_date)
            #contacts = start_simulation(path_pois, scale, detour_radius,contacts)
            #save_results(run_path, scale,detour_radius,contacts)
            contacts, max_blacklist = initialize(path_gps, contacts, granularity,start_date, end_date)
            max_blacklist = max_blacklist +1
            print("contacts passed")
            print("MAX BLACKLIST: ", max_blacklist)
            start_simulation(path_pois, scale, detour_radius,contacts, run_path)
            print("start_simulation passed")
            save_results(run_path, scale,detour_radius,contacts)
            print("save_results passed")
            
#     with open(run_path+"log_engine.csv", 'w') as out:
#         csv_out = csv.writer(out)
#         csv_out.writerow(['bucket', 'type', 'time'])
#         for row in log:
#             csv_out.writerow(row)


In [ ]:
#controllare se il numero di contatti totali è uguale alla dimensione del df number_POI_swap
#statistiche swaps tra solo utenti onesti e attaccanti
df_contacts = pd.read_csv("../../output/distributed_model/test/swap/2009-05-16_2009-05-23/contacts_100_800.csv")
df_number_POI_swap = pd.read_csv("../../output/distributed_model/test/swap/attack2/2009-05-16_2009-05-23/swaps_100_800.csv")
len_POI_swap = len(df_number_POI_swap)
counter_contacts = 0
for index, row in df_contacts.iterrows():
    counter_contacts += row.contacts
print("lunghezza di POI_swap:", len_POI_swap)
print("numero totale di contatti:", counter_contacts)

attackers=[128,30]
#attackers=[30]

a = pd.read_csv("../../output/distributed_model/test/swap/2009-05-16_2009-05-23/swaps_100_800.csv")
#for index, row in a.iterrows():
#    if (row.len_cov_a == 0 and row.len_cov_b == 0):
#        a = a.drop(index)
   # elif (row.MH_a == 0 or row.MH_b == 0):
   #     a = a.drop(index)
mse_a = []
mse_b = []
mh_a = []
ed_a = []
mh_b = []
ed_b = []
for index, row in a.iterrows():
    if (row.POI_node_a == 0 and row.POI_node_b == 0):
        a = a.drop(index)
    elif (row.POI_node_b == 0):
        mh_a.append(row.MH_a)
        ed_a.append(row.ED_a)
        mse_a.append(row.MSE_a)
    elif (row.POI_node_a == 0):
        mh_b.append(row.MH_b)
        ed_b.append(row.ED_b)
        mse_b.append(row.MSE_b)
    else:
        mh_a.append(row.MH_a)
        ed_a.append(row.ED_a)
        mse_a.append(row.MSE_a)
        mh_b.append(row.MH_b)
        ed_b.append(row.ED_b)
        mse_b.append(row.MSE_b)
print("REAL describe")
print("MH_A:")
print(pd.DataFrame(mh_a).describe())
print("count mh_b values:",len(mh_b))
print("MH_B:")
print(pd.DataFrame(mh_b).describe())
print("ED_A:")
print(pd.DataFrame(ed_a).describe())
print("ED_B:")
print(pd.DataFrame(ed_b).describe())
print("MSE_A:")
print(pd.DataFrame(mse_a).describe())
print("MSE_B:")
print(pd.DataFrame(mse_b).describe())
#print(a.describe())
mse_a = []
mse_b = []
mh_a = []
ed_a = []
mh_b = []
ed_b = []
for index, row in df_number_POI_swap.iterrows():
    if (row.POI_node_a == 0 and row.POI_node_b == 0):
        df_number_POI_swap = df_number_POI_swap.drop(index)
    elif (row.node_b not in attackers and row.node_a not in attackers):
        df_number_POI_swap = df_number_POI_swap.drop(index)
    elif (row.node_a in attackers and row.POI_node_a != 0):
        mh_a.append(row.MH_a)
        ed_a.append(row.ED_a)
        mse_a.append(row.MSE_a)
    elif (row.node_b in attackers and row.POI_node_b != 0):
        mh_b.append(row.MH_b)
        ed_b.append(row.ED_b)
        mse_b.append(row.MSE_b)
   # elif (row.MH_a == 0 or row.MH_b == 0):
   #     df_number_POI_swap = df_number_POI_swap.drop(index)

#print(df_number_POI_swap)
#b = df_number_POI_swap.describe()
print("ATTACK describe")
#print(b)

print("MEDIE DELLA MH SOLO PER GLI ATTACCANTI:")
print("count mh_a values:",len(mh_a))
#print(sum(mh_a)/len(mh_a))
print("MH_A:")
print(pd.DataFrame(mh_a).describe())
print("count mh_b values:",len(mh_b))
print("MH_B:")
print(pd.DataFrame(mh_b).describe())
print("ED_A:")
print(pd.DataFrame(ed_a).describe())
print("ED_B:")
print(pd.DataFrame(ed_b).describe())
print("MSE_A:")
print(pd.DataFrame(mse_a).describe())
print("MSE_B:")
print(pd.DataFrame(mse_b).describe())

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#valutazione filtro con confusion matrix e metriche di valutazione

with open(f"{ROOT}/conf.yaml") as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)
out_path = conf['out_path']
#start_date=conf['start_date']
attack_path = conf['attack_path']

#filter_real = pd.read_csv(out_path+"2009-05-01_2009-05-07/filter_100_800.csv")
#filter_attack = pd.read_csv(out_path+"attack/2009-05-01_2009-05-15/filter_100_800.csv")
#print(filter_attack)
out_path = out_path+"swap/"
#start = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
start = "2009-05-24"
#end_date = ["2009-05-07 00:00:00", "2009-05-15 00:00:00"]
#end_date = ["2009-05-15 00:00:00"]
#end_date = ["2009-05-23 00:00:00"]
end_date = ["2009-05-31 00:00:00"]
for end in end_date:
    end =  datetime.strptime(end, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d")
    print(end)
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    print(attack_path+start+"_"+end+"/filter_100_800.csv")
    #print(path_covmap)
    #filter_attack = pd.read_csv(out_path+start+"_"+end+"/filter_100_800.csv")
    filter_attack = pd.read_csv(attack_path+start+"_"+end+"/filter_100_800.csv")
    for index, row in filter_attack.iterrows():
       # print(row.true_label)
        if (row['true_label'] == 1) and (row['filter'] == 1):
            true_positive +=1
        elif (row['true_label']) == 0 and (row['filter'] ==0):
            true_negative +=1
        elif (row['true_label'] == 1) and (row['filter'] ==0):
            false_negative +=1
        else:
            false_positive +=1
    
    print("true positive: ",true_positive)
    print("false positive: ",false_positive)
    print("true negative: ",true_negative)
    print("false negative: ",false_negative)
    
    confusion_matrix_real = confusion_matrix(filter_attack['true_label'],filter_attack['filter'])
    display = ConfusionMatrixDisplay(confusion_matrix_real)
    display.plot()
    plt.show()
    accuracy = accuracy_score(filter_attack['true_label'],filter_attack['filter'])
    precision = (true_positive/(true_positive + false_positive))
    recall = (true_positive/(true_positive + false_negative))
    f1 = f1_score(filter_attack['true_label'],filter_attack['filter'])
    print("accuracy score: ",accuracy)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1 score:", f1)




In [ ]:
#dataset aumentato analisi
import pandas as pd
from pathlib import Path
import yaml
global conf
ROOT = Path().resolve()

with open(f"{ROOT}/conf.yaml") as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)

cov_series_path = conf["cov_series_path"]
start_date=conf['start_date']
ends =conf['end_date']
path_gps = conf['path_gps']
out_path = conf["out_path"]
path_pois = conf["path_pois"]
granularity=conf['granularity']
beijing_lat_min = conf["beijing_lat_min"]
beijing_lat_max = conf["beijing_lat_max"]
beijing_lon_min = conf["beijing_lon_min"]
beijing_lon_max = conf["beijing_lon_max"]

for end_date in ends:
    '''
    print("Initialization started...")
    print("Reading: ../../../dataset/geolife_full_augmented.csv")
    geo = pd.read_csv("../../../dataset/geolife_full_augmented.csv", parse_dates = True)
    
    print("-> Filtering geolife augmented")
    
    geo['date_time'] = pd.to_datetime(geo['date_time'], errors='coerce')
    
    # fiilter with the dates
    geo = geo[(geo.date_time >= start_date) & (geo.date_time <= end_date)]
    
    # filter with the bounding box
    geo = geo[(geo['lat'].between(beijing_lat_min, beijing_lat_max)) & (geo['lon'].between(beijing_lon_min, beijing_lon_max))]
    
    print("LUNGHEZZA GEO AUGMENTED: ",len(geo))

    if granularity == "min":
        geo['date_time'] = geo['date_time'].dt.floor('min')
    if granularity == "h":
        geo['date_time'] = geo['date_time'].dt.floor('h')
    if granularity == "sec":
        geo['date_time'] = geo['date_time'].dt.floor('s')
    
    users = geo['uid'].unique()
    print("ALL USERS: ", users)
    print("NUMBER USERS: ", len(users))
    for user in users:
        print("user : ", user)
        print("count: ", len(geo[geo['uid']==user]))
    '''
    print("start_date: ", start_date)
    print("end_date: ", end_date)
    print("Reading: ../../../dataset/geo_life_full.csv")
    geo = pd.read_csv("../../../dataset/geo_life_full.csv", parse_dates = True)
    
    print("-> Filtering geolife")
    
    geo['date_time'] = pd.to_datetime(geo['date_time'], errors='coerce')
    
    # fiilter with the dates
    geo = geo[(geo.date_time >= start_date) & (geo.date_time <= end_date)]
    
    # filter with the bounding box
    geo = geo[(geo['lat'].between(beijing_lat_min, beijing_lat_max)) & (geo['lon'].between(beijing_lon_min, beijing_lon_max))]
    
    print("LUNGHEZZA GEO: ",len(geo))
    
    
    if granularity == "min":
        geo['date_time'] = geo['date_time'].dt.floor('min')
    if granularity == "h":
        geo['date_time'] = geo['date_time'].dt.floor('h')
    if granularity == "sec":
        geo['date_time'] = geo['date_time'].dt.floor('s')
    
    users = geo['uid'].unique()
    print("ALL USERS: ", users)
    print("NUMBER USERS: ", len(users))
    for user in users:
        print("user : ", user)
        print("count: ", len(geo[geo['uid']==user]))